[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

1 Physical GPUs, 1 Logical GPUs


In [ ]:
# example of training a cyclegan on the horse2zebra dataset
import tensorflow as tf
from random import random
from numpy import load
from numpy import zeros
from numpy import ones
from numpy import asarray
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from matplotlib import pyplot

gpus = tf.config.list_physical_devices('GPU')
gpus
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
# define the discriminator model
def define_discriminator(image_shape):
# weight initialization
 init = RandomNormal(stddev=0.02)
# source image input- lấy ảnh nguồn
 in_image = Input(shape=image_shape)
# C64
 d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
 d = LeakyReLU(alpha=0.2)(d)
# C128
 d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
 d = InstanceNormalization(axis=-1)(d)
 d = LeakyReLU(alpha=0.2)(d)
# C256
 d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
 d = InstanceNormalization(axis=-1)(d)
 d = LeakyReLU(alpha=0.2)(d)
# C512
 d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
 d = InstanceNormalization(axis=-1)(d)
 d = LeakyReLU(alpha=0.2)(d)
# second last output layer
 d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
 d = InstanceNormalization(axis=-1)(d)
 d = LeakyReLU(alpha=0.2)(d)
# patch output
 patch_out = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
# define model
 model = Model(in_image, patch_out)
# compile model
 model.compile(loss='mse', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss_weights=[0.5])
 return model

# generator a resnet block
def resnet_block(n_filters, input_layer):
# weight initialization
 init = RandomNormal(stddev=0.02)
# first layer convolutional layer
 g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(input_layer)
 g = InstanceNormalization(axis=-1)(g)
 g = Activation('relu')(g)
# second convolutional layer
 g = Conv2D(n_filters, (3,3), padding='same', kernel_initializer=init)(g)
 g = InstanceNormalization(axis=-1)(g)
# concatenate merge channel-wise with input layer
 g = Concatenate()([g, input_layer])
 return g

# define the standalone generator model
def define_generator(image_shape, n_resnet=9):
# weight initialization
 init = RandomNormal(stddev=0.02)
# image input
 in_image = Input(shape=image_shape)
# c7s1-64
 g = Conv2D(64, (7,7), padding='same', kernel_initializer=init)(in_image)
 g = InstanceNormalization(axis=-1)(g)
 g = Activation('relu')(g)
# d128
 g = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
 g = InstanceNormalization(axis=-1)(g)
 g = Activation('relu')(g)
# d256
 g = Conv2D(256, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
 g = InstanceNormalization(axis=-1)(g)
 g = Activation('relu')(g)
# R256
 for _ in range(n_resnet):
  g = resnet_block(256, g)
# u128
 g = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
 g = InstanceNormalization(axis=-1)(g)
 g = Activation('relu')(g)
# u64
 g = Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(g)
 g = InstanceNormalization(axis=-1)(g)
 g = Activation('relu')(g)
# c7s1-3
 g = Conv2D(3, (7,7), padding='same', kernel_initializer=init)(g)
 g = InstanceNormalization(axis=-1)(g)
 out_image = Activation('tanh')(g)
# define model
 model = Model(in_image, out_image)
 return model

# define a composite model for updating generators by adversarial and cycle loss- mô hình tổng hợp để cập nhật bộ tạo.
def define_composite_model(g_model_1, d_model, g_model_2, image_shape):
# ensure the model we're updating is trainable
 g_model_1.trainable = True
# mark discriminator as not trainable
 d_model.trainable = False
 # mark other generator model as not trainable
 g_model_2.trainable = False
# discriminator element
 input_gen = Input(shape=image_shape)
 gen1_out = g_model_1(input_gen)
 output_d = d_model(gen1_out)
# identity element
 input_id = Input(shape=image_shape)
 output_id = g_model_1(input_id)
# forward cycle- chu kỳ chuyển tiếp
 output_f = g_model_2(gen1_out)
# backward cycle -chu kì ngược
 gen2_out = g_model_2(input_id)
 output_b = g_model_1(gen2_out)
# define model graph
 model = Model([input_gen, input_id], [output_d, output_id, output_f, output_b])
# define optimization algorithm configuration
 opt = Adam(learning_rate=0.0002, beta_1=0.5)
# compile model with weighting of least squares loss and L1 loss- biên dịch mô hình với tổn thất least squares loss và L1 loss
 model.compile(loss=['mse', 'mae', 'mae', 'mae'], loss_weights=[1, 5, 10, 10], optimizer=opt)
 return model

# load and prepare training images -tải và chuẩn bị hình ảnh đào tạo
def load_real_samples(filename):
# load the dataset
 data = load(filename)
# unpack arrays- giải nén các mảng
 X1, X2 = data['arr_0'], data['arr_1']
# scale from [0,255] to [-1,1]
 X1 = (X1 - 127.5) / 127.5
 X2 = (X2 - 127.5) / 127.5
 return [X1, X2]

# select a batch of random samples, returns images and target -chọn hình ảnh ngẫu nhiên, trả về hình ảnh ,mục tiêu.
def generate_real_samples(dataset, n_samples, patch_shape):
# choose random instances -chọn ảnh
 ix = randint(0, dataset.shape[0], n_samples)
# retrieve selected images -lấy các ảnh đã chọn
 X = dataset[ix]
# generate 'real' class labels (1)- gán nhăn thật bằng 1
 y = ones((n_samples, patch_shape, patch_shape, 1))
 return X, y

# generate a batch of images, returns images and targets-tạo một loạt hình ảnh, trả về hình ảnh, mục tiêu
def generate_fake_samples(g_model, dataset, patch_shape):
# generate fake instance -tạo các ví dụ giả
 X = g_model.predict(dataset)
# create 'fake' class labels (0)-tạo nhãn giả, 0
 y = zeros((len(X), patch_shape, patch_shape, 1))
 return X, y

# save the generator models to file-lưu model bộ tạo thành file
def save_models(step, g_model_AtoB, g_model_BtoA):
# save the first generator model- lưu mô hình GAtoB
 filename1 = 'g_model_AtoB_%06d.h5' % (step+1)
 g_model_AtoB.save(filename1)
# save the second generator model-lưu mô hình GBtoA
 filename2 = 'g_model_BtoA_%06d.h5' % (step+1)
 g_model_BtoA.save(filename2)
 print('>Saved: %s and %s' % (filename1, filename2))

# generate samples and save as a plot and save the model- tạo mẫu và lưu dưới dạng lưu đồ,..
def summarize_performance(step, g_model, trainX, name, n_samples=5):
# select a sample of input images
 X_in, _ = generate_real_samples(trainX, n_samples, 0)
# generate translated images
 X_out, _ = generate_fake_samples(g_model, X_in, 0)
# scale all pixels from [-1,1] to [0,1]
 X_in = (X_in + 1) / 2.0
 X_out = (X_out + 1) / 2.0
# plot real images -ảnh thật
 for i in range(n_samples):
  pyplot.subplot(2, n_samples, 1 + i)
  pyplot.axis('off')
  pyplot.imshow(X_in[i])
# plot translated image -ảnh dịch sang miền khác
 for i in range(n_samples):
  pyplot.subplot(2, n_samples, 1 + n_samples + i)
  pyplot.axis('off')
  pyplot.imshow(X_out[i])
# save plot to file
 filename1 = '%s_generated_plot_%06d.png' % (name, (step+1))
 pyplot.savefig(filename1)
 pyplot.close()

# update image pool for fake images -cập nhật nhóm hình ảnh cho hình ảnh giả mạo
def update_image_pool(pool, images, max_size=50):
 selected = list()
 for image in images:
  if len(pool) < max_size:
   # stock the pool
   pool.append(image)
   selected.append(image)
  elif random() < 0.5:
   # use image, but don't add it to the pool
   selected.append(image)
  else:
   # replace an existing image and use replaced image
   ix = randint(0, len(pool))
   selected.append(pool[ix])
   pool[ix] = image
 return asarray(selected)
# train cyclegan models
def train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset):
# define properties of the training run
 n_epochs, n_batch, = 100, 1 #có thể chọn epochs nhỏ hơn (20 trở lên..)
# determine the output square shape of the discriminator
 n_patch = d_model_A.output_shape[1]
# unpack dataset
 trainA, trainB = dataset
# prepare image pool for fakes
 poolA, poolB = list(), list()
# calculate the number of batches per training epoch
 bat_per_epo = int(len(trainA) / n_batch)
# calculate the number of training iterations
 n_steps = bat_per_epo * n_epochs
# manually enumerate epochs
 for i in range(n_steps):
# select a batch of real samples
  X_realA, y_realA = generate_real_samples(trainA, n_batch, n_patch)
  X_realB, y_realB = generate_real_samples(trainB, n_batch, n_patch)
# generate a batch of fake samples
  X_fakeA, y_fakeA = generate_fake_samples(g_model_BtoA, X_realB, n_patch)
  X_fakeB, y_fakeB = generate_fake_samples(g_model_AtoB, X_realA, n_patch)
# update fakes from pool
  X_fakeA = update_image_pool(poolA, X_fakeA)
  X_fakeB = update_image_pool(poolB, X_fakeB)
# update generator B->A via adversarial and cycle loss
  g_loss2, _, _, _, _  = c_model_BtoA.train_on_batch([X_realB, X_realA], [y_realA, X_realA, X_realB, X_realA])
# update discriminator for A -> [real/fake]
  dA_loss1 = d_model_A.train_on_batch(X_realA, y_realA)
  dA_loss2 = d_model_A.train_on_batch(X_fakeA, y_fakeA)
# update generator A->B via adversarial and cycle loss
  g_loss1, _, _, _, _ = c_model_AtoB.train_on_batch([X_realA, X_realB], [y_realB, X_realB, X_realA, X_realB])
# update discriminator for B -> [real/fake]
  dB_loss1 = d_model_B.train_on_batch(X_realB, y_realB)
  dB_loss2 = d_model_B.train_on_batch(X_fakeB, y_fakeB)
# summarize performance
  print('>%d, dA[%.3f,%.3f] dB[%.3f,%.3f] g[%.3f,%.3f]' % (i+1, dA_loss1,dA_loss2, dB_loss1,dB_loss2, g_loss1,g_loss2))
# evaluate the model performance every so often
  if (i+1) % (bat_per_epo * 1) == 0:
   # plot A->B translation
   summarize_performance(i, g_model_AtoB, trainA, 'AtoB')
   # plot B->A translation
   summarize_performance(i, g_model_BtoA, trainB, 'BtoA')
  if (i+1) % (bat_per_epo * 1) == 0:
   # save the models
   save_models(i, g_model_AtoB, g_model_BtoA)

# load image data
dataset = load_real_samples('horse2zebra_256.npz')
print('Loaded', dataset[0].shape, dataset[1].shape)
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
# generator: A -> B
g_model_AtoB = define_generator(image_shape)
# generator: B -> A
g_model_BtoA = define_generator(image_shape)
# discriminator: A -> [real/fake]
d_model_A = define_discriminator(image_shape)
# discriminator: B -> [real/fake]
d_model_B = define_discriminator(image_shape)
# composite: A -> B -> [real/fake, A]
c_model_AtoB = define_composite_model(g_model_AtoB, d_model_B, g_model_BtoA, image_shape)
# composite: B -> A -> [real/fake, B]
c_model_BtoA = define_composite_model(g_model_BtoA, d_model_A, g_model_AtoB, image_shape)
# train models
train(d_model_A, d_model_B, g_model_AtoB, g_model_BtoA, c_model_AtoB, c_model_BtoA, dataset)



1 Physical GPUs, 1 Logical GPUs
Loaded (1187, 256, 256, 3) (1474, 256, 256, 3)
1/1 [==============================] - 1s 609ms/step
>1, dA[1.344,0.937] dB[1.483,0.888] g[19.919,20.295]
1/1 [==============================] - 0s 20ms/step
>2, dA[12.733,1.725] dB[2.648,0.769] g[17.074,24.996]
1/1 [==============================] - 0s 19ms/step
>3, dA[3.903,1.383] dB[1.807,1.873] g[20.403,19.465]
1/1 [==============================] - 0s 19ms/step
>4, dA[6.708,2.257] dB[4.288,2.135] g[19.907,17.648]
1/1 [==============================] - 0s 18ms/step
>5, dA[7.149,2.324] dB[3.294,3.496] g[19.883,19.843]
1/1 [==============================] - 0s 19ms/step
>6, dA[3.987,1.169] dB[2.615,3.110] g[16.660,17.340]
1/1 [==============================] - 0s 19ms/step
>7, dA[1.252,0.805] dB[1.121,3.736] g[19.590,16.751]
1/1 [==============================] - 0s 19ms/step
>8, dA[1.509,1.000] dB[0.688,1.866] g[15.992,15.183]
1/1 [==============================] - 0s 19ms/step
>9, dA[1.325,1.267] dB[0.74

1/1 [==============================] - 0s 20ms/step
>53, dA[0.287,0.281] dB[0.267,0.186] g[9.202,10.314]
1/1 [==============================] - 0s 19ms/step
>54, dA[0.251,0.275] dB[0.357,0.246] g[10.388,10.171]
1/1 [==============================] - 0s 20ms/step
>55, dA[0.279,0.222] dB[0.359,0.180] g[7.989,7.427]
1/1 [==============================] - 0s 20ms/step
>56, dA[0.337,0.211] dB[0.331,0.323] g[7.909,8.397]
1/1 [==============================] - 0s 25ms/step
>57, dA[0.209,0.228] dB[0.264,0.192] g[8.945,8.828]
1/1 [==============================] - 0s 20ms/step
>58, dA[0.230,0.278] dB[0.388,0.351] g[8.159,7.637]
1/1 [==============================] - 0s 20ms/step
>59, dA[0.238,0.131] dB[0.220,0.307] g[7.130,7.692]
1/1 [==============================] - 0s 20ms/step
>60, dA[0.262,0.142] dB[0.267,0.193] g[9.197,9.739]
1/1 [==============================] - 0s 19ms/step
>61, dA[0.218,0.359] dB[0.247,0.190] g[6.806,7.012]
1/1 [==============================] - 0s 20ms/step
>62, dA[0

1/1 [==============================] - 0s 18ms/step
>157, dA[0.119,0.184] dB[0.134,0.146] g[5.265,5.939]
1/1 [==============================] - 0s 18ms/step
>158, dA[0.207,0.122] dB[0.128,0.145] g[5.466,5.751]
1/1 [==============================] - 0s 19ms/step
>159, dA[0.104,0.076] dB[0.161,0.164] g[8.308,7.191]
1/1 [==============================] - 0s 19ms/step
>160, dA[0.107,0.228] dB[0.179,0.129] g[6.976,7.199]
1/1 [==============================] - 0s 18ms/step
>161, dA[0.110,0.196] dB[0.257,0.157] g[5.381,6.177]
1/1 [==============================] - 0s 19ms/step
>162, dA[0.141,0.050] dB[0.129,0.192] g[7.239,7.014]
1/1 [==============================] - 0s 19ms/step
>163, dA[0.094,0.175] dB[0.120,0.133] g[8.778,9.053]
1/1 [==============================] - 0s 19ms/step
>164, dA[0.108,0.236] dB[0.354,0.125] g[5.569,4.802]
1/1 [==============================] - 0s 18ms/step
>165, dA[0.140,0.074] dB[0.093,0.155] g[8.208,8.533]
1/1 [==============================] - 0s 22ms/step
>16

1/1 [==============================] - 0s 19ms/step
>261, dA[0.278,0.270] dB[0.281,0.257] g[6.896,6.032]
1/1 [==============================] - 0s 19ms/step
>262, dA[0.097,0.312] dB[0.153,0.176] g[7.454,6.988]
1/1 [==============================] - 0s 19ms/step
>263, dA[0.143,0.091] dB[0.121,0.068] g[6.372,7.605]
1/1 [==============================] - 0s 19ms/step
>264, dA[0.215,0.160] dB[0.209,0.048] g[5.510,6.197]
1/1 [==============================] - 0s 19ms/step
>265, dA[0.034,0.414] dB[0.241,0.320] g[5.136,5.069]
1/1 [==============================] - 0s 19ms/step
>266, dA[0.131,0.053] dB[0.050,0.093] g[6.323,6.717]
1/1 [==============================] - 0s 19ms/step
>267, dA[0.190,0.237] dB[0.171,0.071] g[6.370,6.121]
1/1 [==============================] - 0s 21ms/step
>268, dA[0.239,0.184] dB[0.141,0.128] g[5.593,5.678]
1/1 [==============================] - 0s 18ms/step
>269, dA[0.102,0.102] dB[0.188,0.172] g[6.065,6.105]
1/1 [==============================] - 0s 19ms/step
>27

1/1 [==============================] - 0s 19ms/step
>365, dA[0.176,0.091] dB[0.196,0.079] g[5.421,5.926]
1/1 [==============================] - 0s 20ms/step
>366, dA[0.091,0.167] dB[0.109,0.115] g[5.377,5.352]
1/1 [==============================] - 0s 19ms/step
>367, dA[0.299,0.152] dB[0.098,0.163] g[5.398,6.280]
1/1 [==============================] - 0s 19ms/step
>368, dA[0.058,0.037] dB[0.223,0.114] g[6.152,5.858]
1/1 [==============================] - 0s 19ms/step
>369, dA[0.097,0.337] dB[0.205,0.251] g[8.880,7.742]
1/1 [==============================] - 0s 18ms/step
>370, dA[0.109,0.113] dB[0.160,0.107] g[5.490,5.644]
1/1 [==============================] - 0s 20ms/step
>371, dA[0.109,0.153] dB[0.105,0.101] g[6.128,5.820]
1/1 [==============================] - 0s 20ms/step
>372, dA[0.231,0.079] dB[0.093,0.093] g[5.320,5.617]
1/1 [==============================] - 0s 18ms/step
>373, dA[0.131,0.124] dB[0.238,0.189] g[5.358,5.848]
1/1 [==============================] - 0s 18ms/step
>37

...

1/1 [==============================] - 0s 19ms/step
>1350, dA[0.085,0.050] dB[0.298,0.107] g[5.125,4.721]
1/1 [==============================] - 0s 19ms/step
>1351, dA[0.014,0.066] dB[0.033,0.063] g[6.623,7.987]
1/1 [==============================] - 0s 19ms/step
>1352, dA[0.185,0.017] dB[0.215,0.176] g[6.312,7.570]
1/1 [==============================] - 0s 23ms/step
>1353, dA[0.061,0.108] dB[0.016,0.022] g[8.642,7.255]
1/1 [==============================] - 0s 20ms/step
>1354, dA[0.012,0.032] dB[0.300,0.295] g[3.812,3.934]
1/1 [==============================] - 0s 20ms/step
>1355, dA[0.420,0.134] dB[0.086,0.262] g[6.174,5.966]
1/1 [==============================] - 0s 20ms/step
>1356, dA[0.113,0.079] dB[0.117,0.035] g[5.361,6.122]
1/1 [==============================] - 0s 20ms/step
>1357, dA[0.024,0.045] dB[0.524,0.146] g[6.411,5.933]
1/1 [==============================] - 0s 21ms/step
>1358, dA[0.104,0.117] dB[0.177,0.117] g[4.249,4.911]
1/1 [==============================] - 0s 20ms

1/1 [==============================] - 0s 23ms/step
>1402, dA[0.108,0.103] dB[0.548,0.055] g[6.996,7.426]
1/1 [==============================] - 0s 19ms/step
>1403, dA[0.133,0.138] dB[0.074,0.372] g[4.729,5.384]
1/1 [==============================] - 0s 22ms/step
>1404, dA[0.052,0.082] dB[0.091,0.079] g[6.012,5.639]
1/1 [==============================] - 0s 21ms/step
>1405, dA[0.048,0.094] dB[0.301,0.038] g[7.572,8.709]
1/1 [==============================] - 0s 20ms/step
>1406, dA[0.338,0.059] dB[0.040,0.274] g[5.731,6.525]
1/1 [==============================] - 0s 20ms/step
>1407, dA[0.034,0.080] dB[0.079,0.026] g[6.995,7.783]
1/1 [==============================] - 0s 18ms/step
>1408, dA[0.039,0.053] dB[0.348,0.069] g[7.692,7.271]
1/1 [==============================] - 0s 19ms/step
>1409, dA[0.184,0.033] dB[0.195,0.264] g[4.042,4.899]
1/1 [==============================] - 0s 42ms/step
>1410, dA[0.172,0.238] dB[0.103,0.399] g[5.102,5.538]
1/1 [==============================] - 0s 20ms

1/1 [==============================] - 0s 19ms/step
>1454, dA[0.039,0.039] dB[0.240,0.368] g[4.832,5.092]
1/1 [==============================] - 0s 19ms/step
>1455, dA[0.046,0.188] dB[0.068,0.176] g[4.726,5.393]
1/1 [==============================] - 0s 19ms/step
>1456, dA[0.167,0.097] dB[0.198,0.068] g[5.121,5.633]
1/1 [==============================] - 0s 20ms/step
>1457, dA[0.038,0.150] dB[0.043,0.022] g[6.846,7.113]
1/1 [==============================] - 0s 21ms/step
>1458, dA[0.355,0.096] dB[0.326,0.078] g[5.837,5.892]
1/1 [==============================] - 0s 23ms/step
>1459, dA[0.194,0.123] dB[0.025,0.096] g[3.495,4.374]
1/1 [==============================] - 0s 20ms/step
>1460, dA[0.062,0.086] dB[0.156,0.119] g[3.645,3.891]
1/1 [==============================] - 0s 19ms/step
>1461, dA[0.045,0.062] dB[0.019,0.110] g[5.208,4.887]
1/1 [==============================] - 0s 23ms/step
>1462, dA[0.048,0.061] dB[0.042,0.083] g[4.911,5.289]
1/1 [==============================] - 0s 20ms

1/1 [==============================] - 0s 21ms/step
>1506, dA[0.115,0.034] dB[0.206,0.098] g[7.497,6.673]
1/1 [==============================] - 0s 21ms/step
>1507, dA[0.043,0.078] dB[0.027,0.177] g[4.571,4.420]
1/1 [==============================] - 0s 19ms/step
>1508, dA[0.015,0.026] dB[0.090,0.031] g[3.834,3.788]
1/1 [==============================] - 0s 22ms/step
>1509, dA[0.044,0.423] dB[0.358,0.279] g[5.143,5.394]
1/1 [==============================] - 0s 20ms/step
>1510, dA[0.327,0.058] dB[0.184,0.072] g[4.078,4.942]
1/1 [==============================] - 0s 19ms/step
>1511, dA[0.231,0.122] dB[0.073,0.178] g[4.606,5.382]
1/1 [==============================] - 0s 19ms/step
>1512, dA[0.115,0.104] dB[0.040,0.008] g[5.539,6.375]
1/1 [==============================] - 0s 19ms/step
>1513, dA[0.129,0.221] dB[0.391,0.035] g[4.747,4.597]
1/1 [==============================] - 0s 19ms/step
>1514, dA[0.047,0.153] dB[0.201,0.131] g[7.484,7.731]
1/1 [==============================] - 0s 20ms

1/1 [==============================] - 0s 19ms/step
>1558, dA[0.168,0.080] dB[0.014,0.079] g[5.108,5.818]
1/1 [==============================] - 0s 19ms/step
>1559, dA[0.132,0.256] dB[0.225,0.269] g[4.702,4.907]
1/1 [==============================] - 0s 20ms/step
>1560, dA[0.027,0.124] dB[0.364,0.059] g[6.902,8.350]
1/1 [==============================] - 0s 22ms/step
>1561, dA[0.243,0.054] dB[0.074,0.233] g[4.485,5.436]
1/1 [==============================] - 0s 19ms/step
>1562, dA[0.039,0.142] dB[0.058,0.132] g[6.154,6.111]
1/1 [==============================] - 0s 22ms/step
>1563, dA[0.285,0.095] dB[0.352,0.069] g[5.107,4.587]
1/1 [==============================] - 0s 19ms/step
>1564, dA[0.091,0.154] dB[0.038,0.170] g[5.594,5.407]
1/1 [==============================] - 0s 19ms/step
>1565, dA[0.072,0.041] dB[0.243,0.010] g[4.842,5.666]
1/1 [==============================] - 0s 21ms/step
>1566, dA[0.463,0.094] dB[0.096,0.207] g[4.808,4.786]
1/1 [==============================] - 0s 20ms

NameError: name 'load' is not defined